# Exploritory Data Analysis

In this notebook we will explore trends in revenues, expenditures, enrollment, and scores by state over the years.

I. Expenditure

#### Importing packages and data

In [11]:

import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation

import requests
import numpy as np
import plotly.express as px

In [4]:
us_ed = pd.read_csv(r"C:\Users\book_\OneDrive\Desktop\Data_Storage\US_Education\clean_data\state_ed_03_19.csv")
us_ed

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,G08_HI_A_MATHEMATICS,G08_AS_A_READING,G08_AS_A_MATHEMATICS,G08_AM_A_READING,G08_AM_A_MATHEMATICS,G08_HP_A_READING,G08_HP_A_MATHEMATICS,G08_TR_A_READING,G08_TR_A_MATHEMATICS,STATE_ABBREVIATION
0,ALABAMA,2003,727900.0,5196054.0,567704.0,2966981.0,1661369.0,5298932.0,2817111.0,1521462.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
1,ALABAMA,2004,730418.0,5356113.0,625666.0,2971520.0,1758927.0,5450269.0,2875595.0,1580654.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
2,ALABAMA,2005,729342.0,5889966.0,665924.0,3247562.0,1976480.0,5904664.0,3053380.0,1744013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
3,ALABAMA,2006,743265.0,6364028.0,730112.0,3540436.0,2093480.0,6591429.0,3326656.0,1983482.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
4,ALABAMA,2007,743273.0,7069040.0,689072.0,4070949.0,2309019.0,7196459.0,3653466.0,2174210.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,WYOMING,2014,92732.0,1772633.0,112709.0,965213.0,694711.0,1775999.0,865932.0,555761.0,...,275.5,NaN,NaN,248.75,260.0,NaN,NaN,NaN,NaN,WY
812,WYOMING,2015,93867.0,1962874.0,120290.0,1116917.0,725667.0,1942406.0,895910.0,565489.0,...,273.0,NaN,NaN,249.50,251.0,NaN,NaN,NaN,NaN,WY
813,WYOMING,2016,94511.0,2044669.0,123012.0,1175899.0,745758.0,2034229.0,921494.0,585700.0,...,274.0,NaN,NaN,250.25,259.5,NaN,NaN,NaN,NaN,WY
814,WYOMING,2017,94511.0,2044669.0,123012.0,1175899.0,745758.0,2034229.0,921494.0,585700.0,...,275.0,NaN,NaN,251.00,268.0,NaN,NaN,NaN,NaN,WY


## I. Expenditure

In [8]:
fig = px.scatter_geo(data_frame=us_ed, locations='STATE_ABBREVIATION', locationmode='USA-states', animation_frame="YEAR",
                     hover_name="STATE", size="TOTAL_EXPENDITURE",
                     projection="albers usa")
fig.show()

In [66]:
# Funtions for creating animations
pivot = us_ed.pivot(index= 'YEAR', columns='STATE', values="TOTAL_EXPENDITURE")

def prepare_data(df, steps=5):
    df = df.reset_index()
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['YEAR'] = df_expanded['YEAR'].fillna(method='ffill')
    df_expanded = df_expanded.set_index('YEAR')
    df_rank_expanded = df_expanded.rank(axis=1, method='first')
    df_expanded = df_expanded.interpolate()
    df_rank_expanded = df_rank_expanded.interpolate()
    return df_expanded, df_rank_expanded

#Test Run
df_expanded, df_rank_expanded = prepare_data(pivot)

def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]
    

def init():
    ax.clear()
    nice_axes(ax)
    ax.set_ylim(.2, 6.8)

#Need an iterating function for FuncAnimation()
def update(i):
    for bar in ax.containers:
        bar.remove()
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i]
    ax.barh(y=y, width=width, color=colors, tick_label=labels)
    date_str = df_expanded.index[i].strftime('%B %-d, %Y')
    ax.set_title(f'COVID-19 Deaths by Country - {date_str}', fontsize='smaller')
    
fig = plt.Figure(figsize=(4, 2.5), dpi=144)
ax = fig.add_subplot()
anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(df_expanded), 
                     interval=100, repeat=False)

In [68]:
df_expanded.head()

STATE,ALABAMA,ALASKA,ARIZONA,ARKANSAS,CALIFORNIA,COLORADO,CONNECTICUT,DELAWARE,DISTRICT_OF_COLUMBIA,FLORIDA,...,SOUTH_DAKOTA,TENNESSEE,TEXAS,UTAH,VERMONT,VIRGINIA,WASHINGTON,WEST_VIRGINIA,WISCONSIN,WYOMING
YEAR,,,,,,,,,,,,,,,,,,,,,
2003.0,5298932.0,1610289.0,6210287.0,3242799.0,59749885.0,6685029.0,6751815.0,1317447.0,1105963.0,20180142.0,...,993823.0,6482671.0,37333674.0,2957874.0,1440961.0,10746679.0,8802387.0,2524444.0,9128279.0,915515.0
2003.0,5329199.4,1617681.2,6314472.0,3302153.0,60329498.6,6722018.2,6818856.6,1328640.8,1102178.4,20523357.6,...,997477.6,6533857.0,37497029.6,2979636.0,1462800.2,10878154.2,8843905.6,2540354.4,9127892.2,923195.2
2003.0,5359466.8,1625073.4,6418657.0,3361507.0,60909112.2,6759007.4,6885898.2,1339834.6,1098393.8,20866573.2,...,1001132.2,6585043.0,37660385.2,3001398.0,1484639.4,11009629.4,8885424.2,2556264.8,9127505.4,930875.4
2003.0,5389734.2,1632465.6,6522842.0,3420861.0,61488725.8,6795996.6,6952939.8,1351028.4,1094609.2,21209788.8,...,1004786.8,6636229.0,37823740.8,3023160.0,1506478.6,11141104.6,8926942.8,2572175.2,9127118.6,938555.6
2003.0,5420001.6,1639857.8,6627027.0,3480215.0,62068339.4,6832985.8,7019981.4,1362222.2,1090824.6,21553004.4,...,1008441.4,6687415.0,37987096.4,3044922.0,1528317.8,11272579.8,8968461.4,2588085.6,9126731.8,946235.8


### Bar Chart Race Animation

Please refer to the Bar_Chart_Race_Animation notebook for refrence and coding notes, that was used to create this animation

In [70]:
# Creating a function for animated bar graphs

#formatting
def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

def pivoting(measure, df=us_ed, index='YEAR', column='STATE'):
    pivot_table = df.pivot(index= index, columns=column, values=measure)
    return pivot_table
    
def prepare_data(df, steps=5):
    df = df.reset_index()
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['YEAR'] = df_expanded['YEAR'].fillna(method='ffill')
    df_expanded = df_expanded.set_index('YEAR')
    df_rank_expanded = df_expanded.rank(axis=1, method='first')
    df_expanded = df_expanded.interpolate()
    df_rank_expanded = df_rank_expanded.interpolate()
    return df_expanded, df_rank_expanded

def init():
    ax.clear()
    nice_axes(ax)
    ax.set_ylim(.2, 6.8)

# Iterating funcition for funcAnimation
def update(i):
    for bar in ax.containers:
        bar.remove()
    y = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i]
    ax.barh(y=y, width=width, color=colors, tick_label=labels)
    # date_str = df_expanded.index[i].strftime('%B %-d, %Y')
    ax.set_title(f'US Education Total Expendature by State - {df_expanded.index[i]}', fontsize='smaller')


# Step 1: pivot table to hold necessary data
def bar_animation(measure, df=us_ed, index='YEAR', column='STATE'):
    fig, ax = plt.subplots(figsize=(8, 20), dpi=144)
    colors = sns.color_palette("hls", 51)
    y = s.index
    width = s.values
    
    fig = plt.Figure(figsize=(10, 20), dpi=144)
    ax = fig.add_subplot()
    anim = FuncAnimation(fig=fig, func=update, frames=len(df_expanded), 
                     interval=100, repeat=False)
    HTML(anim.to_jshtml())

In [ ]:
#formatting
fig, ax = plt.subplots(figsize=(8, 20), dpi=144)
colors = sns.color_palette("hls", 51)
y = s.index
width = s.values

#data preparation
df_expanded, df_rank_expanded = prepare_data(df)
labels = df_expanded.columns

In [71]:
bar_animation(measure= "TOTAL_EXPENDITURE")
plt.show()

In [49]:
pivot = us_ed.pivot(index= 'YEAR', columns='STATE', values="TOTAL_EXPENDITURE")
pivot.loc[2003]

STATE
ALABAMA                  5298932.0
ALASKA                   1610289.0
ARIZONA                  6210287.0
ARKANSAS                 3242799.0
CALIFORNIA              59749885.0
COLORADO                 6685029.0
CONNECTICUT              6751815.0
DELAWARE                 1317447.0
DISTRICT_OF_COLUMBIA     1105963.0
FLORIDA                 20180142.0
GEORGIA                 13516783.0
HAWAII                   1602392.0
IDAHO                    1726941.0
ILLINOIS                21278177.0
INDIANA                  9777720.0
IOWA                     4517990.0
KANSAS                   3818740.0
KENTUCKY                 4700494.0
LOUISIANA                5515322.0
MAINE                    2154117.0
MARYLAND                 8732518.0
MASSACHUSETTS           11382398.0
MICHIGAN                19680370.0
MINNESOTA                8938770.0
MISSISSIPPI              3205193.0
MISSOURI                 7905481.0
MONTANA                  1214879.0
NEBRASKA                 2623966.0
NEVADA        